In [2]:
#Bibliotecas
import numpy as np
import random
import time

In [4]:
def dibujar_batalla_barcos():
    escena = [
        "      __|__                     __|__      ",
        "     |o o o|                   |o o o|     ",
        "  ___/_____/\\__             ___/_____/\\__   ",
        "  \\           /             \\           /   ",
        "~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~",
        "               #-------->>                    ",
        "                BOOOOOM!                      ",]
    for linea in escena:
        print(linea)

dibujar1 = dibujar_batalla_barcos()

      __|__                     __|__      
     |o o o|                   |o o o|     
  ___/_____/\__             ___/_____/\__   
  \           /             \           /   
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
               #-------->>                    
                BOOOOOM!                      


In [56]:
def crear_tablero(tamaño):
    tablero = np.full((tamaño,tamaño), "_")
    return tablero

tablerillo = crear_tablero(10)
print(tablerillo)

tablerillo1 = crear_tablero(10)



[['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']]


In [27]:
def comprobar_colision(barco, tablero): 
    for i, j in barco:
        if tablero[i][j] == 'O':  
            return True
    return False                    

#False = no hay colision, True = si hay colision

False


In [49]:
def crear_barco(eslora, tablero):
    while True:
        casilla_0 = (random.randint(0, 9), random.randint(0, 9))
        orientacion = random.choice(["Vertical", "Horizontal"])

        barco = [casilla_0]
        casilla = casilla_0

        for _ in range(1, eslora): #_sifnifica que se va iterar tantas veces se necesite, es una convencion de python
            if orientacion == "Vertical":
                nueva_casilla = (casilla[0] + 1, casilla[1])
            else:
                nueva_casilla = (casilla[0], casilla[1] + 1)

            if nueva_casilla[0] > 9 or nueva_casilla[1] > 9:
                break  
            barco.append(nueva_casilla)
            casilla = nueva_casilla

        if len(barco) == eslora: 
            if not comprobar_colision(barco, tablero):
                return barco
            
barquilla = crear_barco(6,tablerillo)
print(barquilla)

comprobasao = comprobar_colision(barquilla, tablerillo)
print(comprobasao)

[(9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 9)]
False


In [57]:
def crear_barco_modo_corto(tablero):
    barcos=[]
    for _ in range(6):
        barco = [(random.randint(0, 9), random.randint(0, 9))]
        while comprobar_colision(barco, tablero):
            barco = [(random.randint(0, 9), random.randint(0, 9))]
        barcos.append(barco)
        colocar_flota([barco], tablero)
    return 

barquitos = crear_barco_modo_corto(tablerillo)
print(barquitos)

None


In [54]:
def crear_flota(tablero):
    flota_barcos = [
        crear_barco(2, tablero),
        crear_barco(2, tablero),
        crear_barco(2, tablero),
        crear_barco(3, tablero),
        crear_barco(3, tablero),
        crear_barco(4, tablero)]
    return flota_barcos

flotilla = crear_flota(tablerillo)
print(flotilla)

[[(0, 7), (1, 7)], [(1, 5), (2, 5)], [(2, 5), (3, 5)], [(7, 6), (7, 7), (7, 8)], [(6, 7), (6, 8), (6, 9)], [(1, 5), (2, 5), (3, 5), (4, 5)]]


In [67]:
def colocar_flota(flota, tablero): 
    for barco in flota:
        for i, j in barco:
            tablero[i][j] = 'O'  
    return tablero

def colocar_barcos_modo_corto(flota, tablero): 
    for x in flota:
        tablero[i][j] = 'O'  
    return tablero

colocada = colocar_flota(flotilla,tablerillo)
print(colocada)

[['_' '_' '_' '_' '_' '_' '_' 'O' '_' '_']
 ['_' 'O' '_' '_' '_' 'O' '_' 'O' '_' '_']
 ['_' '_' '_' '_' '_' 'O' '_' 'O' '_' '_']
 ['_' '_' '_' '_' '_' 'O' '_' '_' '_' '_']
 ['_' '_' 'O' '_' '_' 'O' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' '_' '_' '_']
 ['_' '_' '_' '_' '_' '_' '_' 'O' 'O' 'O']
 ['_' '_' '_' '_' '_' '_' 'O' 'O' 'O' '_']
 ['_' 'O' '_' '_' '_' '_' '_' '_' '_' 'O']
 ['_' '_' '_' 'O' '_' '_' '_' '_' '_' '_']]


In [68]:
colocada = colocar_barcos_modo_corto(barquitos,tablerillo1)
print(colocada)

TypeError: 'NoneType' object is not iterable

In [ ]:
def generar_nuevo_tablero():
    tablero_usuario = crear_tablero(10)
    flota_usuario = crear_flota(tablero_usuario)
    tablero_final = colocar_flota(flota_usuario, tablero_usuario)
    return tablero_final

In [ ]:
def sistema_de_turnos(tablero_final, tablero_maquina):
    print("Que comience el juego")
    puntos_usuario = 0
    puntos_maquina = 0
    if modo == "corto":
        limite_puntos = 2
    else:
        limite_puntos = len(tablero_final[tablero_final == "O"])
    
    while puntos_usuario < limite_puntos and puntos_maquina < limite_puntos:
        print("\n--- Tu turno ---")    
        puntos_usuario += turno_usuario(tablero_maquina)
        
        if puntos_usuario > limite_puntos:   
            print("Has ganado makena.")
            break
        time.sleep(1)

        print("\n--- Turno de la Máquina ---")   
        puntos_maquina += turno_maquina(tablero_final)
        print(tablero_final)

        if puntos_maquina >= limite_puntos:      
            print("La máquina te ha ganado crack")
            break


In [ ]:
def verificar_victoria(tablero_oponente,modo):
    if modo == "Corto":
        barcos_restantes = len(np.where(tablero_oponente == "O")[0])
        barcos_hundidos = 6 - barcos_restantes
        return barcos_hundidos >= 2 #Gana si almenos se hunden dos barcos
    else:
        return not np.any(tablero_oponente == "O") #Gana si no queda ningun barco

In [ ]:
def turno_usuario(tablero):
    while True:  
        try:                                                                 #Mientras no se pierda el turno
            fila = int(input("Introduce la fila (1-9) a la que quieres disparar: "))
            columna = int(input("Introduce la columna (1-9) a la que quieres disparar: "))
            if not (0 <= fila <= 9) or not columna (0 <= columna <= 9):
                print("Cordenadas fuera de rango, intentelo de nuevo")
                continue
        except ValueError:
            print("Intruce numero entre el 1 y 9.")
            continue

        resultado = disparar((fila-1, columna-1), tablero)             #llamada ala función disparar
        print(f"Disparaste a {(fila, columna)}:\n {resultado}")     #Ubicación del tiro por pantalla
        
        if resultado == "Agua":                                     #Posibilidades
            print("Fallaste. Fin de tu turno.")
            return 0
        elif resultado == "Tocado":
            print("Has acertado sigue disparando.")
            if barco_hundido(tablero):
                print("Barco Hundido")
                return 1
            

In [ ]:
def turno_maquina(tablero):
    while True:                      #Mientras acierte seguirá disparando de forma random
        fila = random.randint(0, 9)
        columna = random.randint(0, 9)
        
        resultado = disparar((fila, columna), tablero)
        print(f"La máquina disparó a {(fila, columna)}: \n {resultado}")
        
        if resultado == "Agua":
            print("La máquina falló. Fin de su turno.")
            return 0  

        elif resultado == "Tocado":
            print("La máquina acertó. Sigue disparando.")
            return 1

In [ ]:
def disparar(casilla, tablero):
    fila, columna = casilla
    if tablero[fila, columna] == "O":              #la O es barco, la X tocado y la A agua
        print("Tocado")
        tablero[fila, columna] = "X"
        return "Tocado"
    elif tablero[fila,columna] == '_': 
        print("Agua")
        tablero[fila,columna] = "A"
        return "Agua"
    else:
        print("Casilla ya disparada.")
        return "Casilla repetida"

In [ ]:
def barco_hundido(tablero,fila,columna):
    return not np.any(tablero == "O")

In [ ]:
def jugar():
    print(f"Welcome to Battleship")
    time.sleep(1)

    dibujar_batalla_barcos()

    time.sleep(2)

    modo = input("Hay dos modos de juego: Corto( Cada jugador tendra 6 barcos de eslora 1 y quién elimine\
                 \ antes dos barcos habrá ganado. Largo: 6 barcos cada uno y quien elimine los del\
                  contrincante antes habrá ganado.  Introduzca: (Corto/Largo)) ").lower()

    print(f"Creating your new the game track...")

    time.sleep(1)
    tablero_usuario = crear_tablero(10)
    tablero_maquina = crear_tablero(10)

    
    if modo == "corto":
        flota_usuario = crear_barcos_modo_corto(tablero_usuario)
        flota_maquina = crear_flota(tablero_maquina)
    else:
        flota_usuario = crear_flota(tablero_usuario)
        flota_maquina = crear_flota(tablero_maquina)
        #print(flota_usuario) if you want to see yours boats before placing in the game track

    time.sleep(0.5)
    print(f"Your fleet will be made up of 6 ships and they will be placed randomly on the game track.")


    tablero_final = colocar_flota(flota_usuario,tablero_usuario)
    tablero_maquina_final = colocar_flota(flota_maquina,tablero_maquina)
    print(f"\n               Here you are")
    print(tablero_final)

    while True: 
        respuesta = input("Te gusta tu tablero o generamos otro?(True/False)")    
        if respuesta == "True":
            print(f"Then you will play with this, GOOD LUCK!")
            break
        else:
            print("Generando nuevo tablero")
            time.sleep(1)
            tablero_final = generar_nuevo_tablero()
            print(tablero_final)
        
    sistema_de_turnos(tablero_final, tablero_maquina)


In [ ]:
def hundir_la_flota():
    while True:
        jugar()
        jugar_nuevamente = input("¿Quieres jugar de nuevo? (True/False): ").lower()
        if jugar_nuevamente != "True":
            print(f"Saliendo del juego, que tenga buen día no dude en volver a jugar")
            print(f"Con un pull de git compruebe las nuevas actualizaciones")
            break

In [ ]:
hundir_la_flota()